In [379]:
import pytesseract
import cv2
from PIL import Image 
import re
import pandas as pd

In [380]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [391]:
formats = pd.read_excel('invoice_formats.xlsx')

In [399]:
%%time
dat = {
    "Vat Number":'',
       "Total":0,
       "Date":'',
      "Invoice_Number":''}
img  = cv2.imread('Makro Invoices/Makro Invoices/New/04.02.20_Cafes Careca SL_474,49_page1.png', cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
vat = []
print(type(text_list))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):


        if (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            print(b)
        elif (re.search("[0-9]{8}-[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}-[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)

    
        elif (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)

        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            
        elif (re.search("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)
            dat["Vat Number"] = b[0]

        if len(vat)>1:
            dat["Vat Number"] = vat[0]
        else:
            pass
         #Date section           
        if (re.search("\d{2}/\d{2}/\d{4}",a)):
            b  = re.findall("\d{2}/\d{2}/\d{4}", a)
            dat["Date"] = b[-1]
        if (re.search("\d{2}-\d{2}-\d{4}",a)):
            b = re.findall("\d{2}-\d{2}-\d{4}",a)
            dat["Date"] = b[0]
        if (re.search("\d{2}/\d{2}/\d{2}",a)):
            b = re.findall("\d{2}/\d{2}/\d{2}",a)
            dat["Date"] = b[0]

        #invoice number section
        if ("Factura" in a or "ALBARAN" in a or "Factura: " in a):

            print(a)
            b = a.split(" ")
            if b[0] == "ALBARAN":
                dat["Invoice_Number"] = b[1]
            print(b)
            if len(b)>3:
                print(b)
                if len(b)>4 and b[2] !='—' and len(b[1])!=25 and len(b[1])>5:
                    dat["Invoice_Number"] = b[1]+b[2]
                    print(b)
                elif len(b)>=5 and len(b[1])==25:
                    dat["Invoice_Number"] = b[1]
                elif len(b)>6 and len(b[3])>15:
                    dat["Invoice_Number"] = b[1]+ b[2]+ b[3]
                elif len(b)>6 and b[1]== "0/0":
                    dat["Invoice_Number"] = b[1] + b[2]
                    
        if ("Tarjeta" in a or "Credito" in a or  "pagina" in a or "Total:" in a  or "TOTAL" in a):
            print("Case Total")
            b = a.split(" ")
            if len(b[-1])==2:
                
                dat["Total"] = b[-2]+b[-1]
                print(b[-2]+b[-1])
            if b[-1]=='€':
                dat["Total"] = b[-2]
            else:
                dat["Total"] = b[-1]
                print(b[-1])
        

<class 'list'>
A pagar el 4/02/2020 © 474,49€por —_—_Contado Total Factura: 474,49
['A', 'pagar', 'el', '4/02/2020', '©', '474,49€por', '—_—_Contado', 'Total', 'Factura:', '474,49']
['A', 'pagar', 'el', '4/02/2020', '©', '474,49€por', '—_—_Contado', 'Total', 'Factura:', '474,49']
['B-38360473']
Wall time: 9.35 s


In [400]:
#data = pd.DataFrame.from_dict(dat)
data['Vat Number']= data['Vat Number'].str.replace(' ', '')
print(dat)

{'Vat Number': ['B76744861'], 'Total': 0, 'Date': '31/00/21', 'Invoice_Number': ''}


In [401]:
bp = dat

In [402]:
data = pd.DataFrame(bp,index=[0])

In [403]:
data['Vat Number']= data['Vat Number'].str.replace('-', '')
data['Vat Number']= data['Vat Number'].str.replace(',', '')
data['Vat Number']= data['Vat Number'].str.replace('.', '')
data

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


,Vat Number,Total,Date,Invoice_Number
0,B76744861,0,31/00/21,


In [404]:
formats['last']= (formats['nif'].isin(data['Vat Number']))

In [405]:
df2 = formats.query("last == True")

In [406]:
df2

,formato,nombre,nif,last
9419,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
12062,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
12178,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
12780,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
16962,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
17485,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
18171,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
18852,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
18939,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
20388,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True


In [407]:
inv = 0
for i in df2['formato']:
    inv = i
pat = 'F1-1-01353'
factura = re.compile(inv)
mo = factura.search(text)
dat["Invoice_Number"]=mo.group()

In [408]:
dat

{'Vat Number': ['B76744861'],
 'Total': 0,
 'Date': '31/00/21',
 'Invoice_Number': '76744861'}

In [409]:
finals  =  pd.DataFrame(dat,index=[0])

In [410]:
final = final.append(finals)

In [411]:
final

,Vat Number,Total,Date,Invoice_Number
0,A-38003414,"90,21",30/06/2020,1600359177
0,A-38003414,"154,05",27-05-2021,1400804213
0,G-38055612,"49,52",01/07/2020,2020-00116355
0,B76546787,siguiente,02/08/2021,721000050614
0,B76744861,"www.gmfood.es,",03/02/2020,200302180004999
0,B38574281,TOTAL,03/07/2020,F1-1-01353
0,"B-76,305.333",0,01/11/2020,2020/129
0,A30013841,0,03/05/2022,1303205930
0,78548857-Q,FACTURA,01/08/2021,19/2021
0,G-38055612,"49,52",01/07/2020,2020-00116355


In [412]:
final.to_excel('Extracted_Data.xlsx')

In [398]:
text

'INVERMIRA,S.L.\nB38323812\nCtra.-TF 653 - N° 50\n38632 ARONA - TENERIFE\nTelf. 922732587\nFax. 922786404\nwww.invermira.com\ninvermira@invermira.com\n\nFACTURA\n\nFACTURA | 3-92002517 1 Giiente 001000006\nFecha 03/11/2020 VIVEROS MOGAN S.L.\n\nCTRA. POSTREAGUA, S/N PUERTO DE MOGAN\n\n35138 MOGAN, DE\nPAGARE NO ENTREGADO PALMAS , LAS\n\nESPANA\n[observaciones:\n\nTel. 928565434\n\nCIF.B76305333\n[Atticuto | ~ Desoripeién |) Unidades || Precio || Dto |) % gic || Importe |\nAlbaran 3-20.002.980 de Fecha 03/11/2020\n000000665 GERANIO P. M/10.5 325,00 0,6893 3,00 224,03\n000000314 TOMILLO M/2.5 3,00 2,2505 3,00 6,75\nP 000000334 CILANTRO M/10.5, 24,00 0,7379 3,00 17,71\n000000318 ROMERO M/2.5 29,00 2.4272 3,00 70,39\n000000346 CURRY M/10,5 24,00 0,7379 3,00 17,71\nP 000000447 PIMIENTO DULCE HUERTO URB M/ 17 5,00 1,5922 3,00 7,96\nP 000000441 PIMIENTO PADRON M/ 17 5,00 1,5922 3,00 7,96\nP 000000448 PIMIENTA PICANTE H. URB M/ 17, 5,00 1,5922 3,00 7,96\n000000237 CONVOLVULUS M/10.5 24,00 1,03